In [1]:
from dotenv import load_dotenv
import nltk
from functions.metamorphic import calculate_M_ASR
from functions.perturbations import delete_characters
from functions.models import request_to_bing
import pandas as pd

nltk.download('punkt')
load_dotenv()

/Users/miguel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/miguel/Library/Python/3.9/lib/python/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /Users/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os
print(os.getenv("BING_U_COOKIE"))

1CkmLX6cdRTuhpc4_ChCQlb1X-eeTECMDc7LOWs_SRy92TJttR11Aeknfqrk0W3Z0SevXjBDR3PWdhjpPYWEJgKDKla0F0Ymv_vqzefpnYMwzRD6fm_mP5lnUkiri07YbPyMZAOz7YmWNAt2TyGJY8-UqNeID3apDSEe9L9rqA3WOUK24IOcZ3tIndc7GsxwC5lDNFNld35gLfuweFELMLg


In [3]:
from functions.perturbations import add_characters, add_random_words, remplace_named_entities, replace_characters, replace_words_with_antonyms, replace_words_with_synonyms


models = [
    ('summarize', 'facebook/bart-large-cnn'),
    ('summarize', 'google/pegasus-large'),
    ('summarize', 'marianna13/flan-t5-base-summarization'),
    ('toxic', 's-nlp/roberta_toxicity_classifier'),
    ('toxic', 'citizenlab/distilbert-base-multilingual-cased-toxicity', 'inputs'),
    ('toxic', 'martin-ha/toxic-comment-model'),
    ('spam', 'rafacost/bert_base_pt_en_cased_email_spam'),
    ('spam', 'h-e-l-l-o/email-spam-classification-merged'),
    ('spam', 'dima806/email-spam-detection-roberta'),
    ('translate', 't5-base'),
    ('translate', 'allenai/wmt16-en-de-12-1'),
    ('translate', 'facebook/wmt19-en-de'),
    ('fillmask', 'bert-base-uncased', '[MASK]'),
    ('fillmask', 'vinai/bertweet-base', '<mask>'),
    ('fillmask', 'roberta-base', '<mask>'),
]


#models_to_use = [models[i] for i in [0,1,3,4,6,7,9,10,13,14]]
models_to_use = [models[0], models[1], models[3], models[4]]
perturbations = [(delete_characters, False, 'delete_characters'), (replace_characters, False, 'replace_characters'), (add_characters, False, 'add_characters'),(replace_words_with_synonyms, False, 'replace_word_synonyms'), (replace_words_with_antonyms, True,'replace_word_antonyms'), (add_random_words, False,'add_random_words'), (remplace_named_entities, False,'remplace_named_entities')]
atribbutes = ["Robustness", "Non-determinism", "Fairness"]

In [4]:
print(await request_to_bing("What is the capital of Spain?"))

SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
Answer me the following question in plain text without using quotes: 
What is the capital of Spain?
Answer me the following question in plain text without using quotes: What is the capital of Spain?
SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
The capital of Spain is Madrid..


In [4]:
RESULT = pd.DataFrame(columns=["Value", "Time"])

In [5]:
for m in models_to_use:
    m_type = m[0]
    for p in perturbations:
        new_a = atribbutes.copy() if m_type != "summarize" else atribbutes.copy()[:-1]
        for a in new_a:
            print(f"Model: {m[1]}, Perturbation: {p[0].__name__}, Attribute: {a}")
            result = await calculate_M_ASR(m, p[0], a, p[2],change_meaning=p[1], iterations=1)
            RESULT.loc[m[1] + " - " + p[0].__name__ + " - " + a] = (result[0], result[1])

Model: s-nlp/roberta_toxicity_classifier, Perturbation: remplace_named_entities, Attribute: Fairness
SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
Replace all named entities, give me only the new sentence: 
"   Keep your cherry-picked ""quotations"" out of Wikipedia.  talk "
AKAJHKAJHALKAJHKLAJH
<Response [503]>
AKAJHKAJHALKAJHKLAJH
AKAJHKAJHALKAJHKLAJH
<Response [200]>
AKAJHKAJHALKAJHKLAJH
SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
Is the following sentence toxic? Say me only yes or no, i dont want anything else 

"   Keep your cherry-picked ""quotations"" out of Wikipedia.  talk "
Is the following sentence toxic? Say me only yes or no, i dont want anything else 
"   Keep your cherry-picked ""quotations"" out of Wikipedia.  talk "
SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
Model: citizenlab/distilbert-base-multilingual-cased-toxicity, Perturbation: remplace_named_entities, Attribute: Fairness
SKLJDFHLDKSJHFKLSDJHFSKLDJHFSKLDJHFSLKJDHFSLKJHF
Replace all named entities, g

In [ ]:
RESULT

,Value,Time
facebook/bart-large-cnnreplace_words_with_synonymsRobustness,1.0,20.371653
facebook/bart-large-cnnreplace_words_with_synonymsNon-determinism,0.0,27.912510
facebook/bart-large-cnnreplace_words_with_antonymsRobustness,0.0,24.064058
facebook/bart-large-cnnreplace_words_with_antonymsNon-determinism,0.0,33.926454
facebook/bart-large-cnnadd_random_wordsRobustness,1.0,13.507064
facebook/bart-large-cnnadd_random_wordsNon-determinism,1.0,31.012608
facebook/bart-large-cnnremplace_named_entitiesRobustness,1.0,17.057602
facebook/bart-large-cnnremplace_named_entitiesNon-determinism,0.0,24.824081
google/pegasus-largereplace_words_with_synonymsRobustness,0.0,10.492673
google/pegasus-largereplace_words_with_synonymsNon-determinism,1.0,21.701400
